# Анализ SocialFX и AutoEQ для выбора частот эквалайзера

## Цель анализа

Цель данного анализа — сравнить два источника данных эквализации:

- SocialFX — реальные пользовательские настройки эквалайзера
- AutoEQ — алгоритмическая компенсация амплитудно-частотной характеристики

Основные задачи анализа:

- определить наиболее важные частотные зоны
- сравнить пользовательскую практику и алгоритмическую коррекцию
- сформировать обоснованный набор частот эквалайзера
- проверить согласованность результатов со стандартами (ISO)

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans

DATA_DIR = Path("../../datasets")
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# SocialFX
wimp = pd.read_csv(DATA_DIR / "socialfx_analysis/socialfx_wimp.csv")
eq_long_w  = pd.read_csv(DATA_DIR / "socialfx_analysis/socialfx_eq_long_with_w.csv")

# AutoEQ (после твоего "clean" препроцесса)
autoeq = pd.read_csv(DATA_DIR / "autoeq_analysis/autoeq_parametric_eq_clean.csv")

# ----------------------
# Standard frequencies (ISO-ish octave bands)
# ----------------------
ISO_STD = np.array([31.5, 63, 125, 250, 500, 1000, 2000, 4000, 8000, 16000], dtype=float)

def normalize_01(y):
    y = np.asarray(y, dtype=float)
    y = np.where(np.isfinite(y), y, np.nan)
    mn = np.nanmin(y)
    mx = np.nanmax(y)
    return (y - mn) / (mx - mn + 1e-12)

def interp_logx(x_src, y_src, x_new, fill_mode="edge"):
    """
    Interpolate y(x) onto x_new in log10(x) space.
    fill_mode: "edge" keeps edge values, "zero" sets out-of-range to 0.
    """
    x_src = np.asarray(x_src, dtype=float)
    y_src = np.asarray(y_src, dtype=float)
    x_new = np.asarray(x_new, dtype=float)

    order = np.argsort(x_src)
    x_src = x_src[order]
    y_src = y_src[order]

    lx_src = np.log10(x_src)
    lx_new = np.log10(x_new)

    if fill_mode == "zero":
        y_new = np.interp(lx_new, lx_src, y_src, left=0.0, right=0.0)
    else:
        y_new = np.interp(lx_new, lx_src, y_src, left=y_src[0], right=y_src[-1])
    return y_new


## Базовая частотная сетка (ISO-ish octave bands)

В качестве общей частотной сетки используется набор частот из датасета SocialFX.

Причины выбора:

- отражает реальные решения пользователей
- частоты распределены логарифмически
- сетка достаточно плотная для анализа
- удобно использовать как основу для сравнения с AutoEQ

### Build SocialFX curves on grid

In [4]:
grid_fc = np.sort(wimp["fc_hz"].unique())

# SocialFX: wimp = mean(|gain| * consistency) уже посчитан
social_wimp = wimp.rename(columns={"w": "social_wimp"}).copy()

# SocialFX: mean(|gain|) (без consistency)
social_abs = (
    eq_long_w.dropna(subset=["gain_db", "fc_hz"])
             .assign(abs_gain=lambda d: d["gain_db"].abs())
             .groupby("fc_hz")["abs_gain"]
             .mean()
             .reset_index()
             .rename(columns={"abs_gain": "social_abs_gain"})
)

social = (pd.DataFrame({"fc_hz": grid_fc})
          .merge(social_wimp, on="fc_hz", how="left")
          .merge(social_abs, on="fc_hz", how="left")
          .sort_values("fc_hz"))

social.head()


,fc_hz,social_wimp,social_abs_gain
0,20.0,0.522460,1.116843
1,50.0,0.569269,1.193761
2,83.0,0.603042,1.246820
3,120.0,0.597699,1.226838
4,161.0,0.582495,1.189631


## Метрики SocialFX

Для анализа SocialFX используются две метрики.

Основная метрика:

- используется модуль усиления (abs(gain))
- значение умножается на ratings_consistency
- результат усредняется по каждой частоте

Данная метрика отражает:
- насколько часто пользователи вмешиваются в частоту
- насколько они уверены в этих решениях

Дополнительная контрольная метрика:

- используется среднее значение abs(gain) без учёта consistency
- применяется для проверки устойчивости результатов

### Build AutoEQ intensity curve + project to grid

## Метрика AutoEQ

Для AutoEQ используется метрика:

- среднее значение abs(gain) по параметрическим фильтрам

Значение Preamp не учитывается, поскольку:

- оно не связано с конкретной частотой
- применяется одинаково ко всему спектру
- используется исключительно для предотвращения клиппинга

## Проекция AutoEQ на частотную сетку SocialFX

Частоты AutoEQ не совпадают с частотами SocialFX, поэтому выполняется интерполяция.

Особенности интерполяции:

- интерполяция проводится в логарифмическом пространстве частот
- сохраняется форма распределения интенсивности
- позволяет корректно сравнивать кривые по форме

In [5]:
autoeq_fc_abs = (
    autoeq.dropna(subset=["gain_db", "fc_hz"])
         .assign(abs_gain=lambda d: d["gain_db"].abs())
         .groupby("fc_hz")["abs_gain"]
         .mean()
         .reset_index()
         .rename(columns={"abs_gain": "autoeq_abs_gain"})
         .sort_values("fc_hz")
)
# AutoEQ — это не «мнение», а алгоритмическая компенсация нас интересует не знак, а насколько сильно алгоритм вынужден править частоту
# Поэтому abs(gain_db) здесь абсолютно корректен

# Interpolate onto SocialFX grid (log-frequency interpolation) Мы делаем интерполяцию, чтобы сравнение было по форме, а не по сетке.
autoeq_on_grid = interp_logx(
    autoeq_fc_abs["fc_hz"].values,
    autoeq_fc_abs["autoeq_abs_gain"].values,
    grid_fc,
    fill_mode="edge"
)

autoeq_grid = pd.DataFrame({"fc_hz": grid_fc, "autoeq_abs_gain": autoeq_on_grid})

cmp = (social.merge(autoeq_grid, on="fc_hz", how="left")
             .sort_values("fc_hz"))

cmp.head()


,fc_hz,social_wimp,social_abs_gain,autoeq_abs_gain
0,20.0,0.522460,1.116843,0.904167
1,50.0,0.569269,1.193761,4.886905
2,83.0,0.603042,1.246820,3.304286
3,120.0,0.597699,1.226838,2.307246
4,161.0,0.582495,1.189631,3.264198


## Нормализация данных

Поскольку SocialFX и AutoEQ имеют разные масштабы усилений, все кривые нормализуются в диапазон от 0 до 1.

Нормализация позволяет:

- сравнивать относительную важность частот
- игнорировать различия в абсолютных значениях dB
- сосредоточиться на форме распределения

## Усреднённая кривая важности

Для объединения источников используется усреднённая кривая:

- 50 процентов SocialFX
- 50 процентов AutoEQ

Назначение усреднённой кривой:

- объединить пользовательскую практику и алгоритмическую коррекцию
- получить рабочий prior для выбора частот
- использовать как основу для кластеризации

In [6]:
cmp2 = cmp.copy()

cmp2["social_wimp_norm"] = normalize_01(cmp2["social_wimp"])
cmp2["social_abs_norm"]  = normalize_01(cmp2["social_abs_gain"])
cmp2["autoeq_abs_norm"]  = normalize_01(cmp2["autoeq_abs_gain"])

# Mean curve: equal weights (можно поменять позже)
w_social = 0.5
w_auto   = 0.5
cmp2["mean_norm"] = w_social * cmp2["social_wimp_norm"] + w_auto * cmp2["autoeq_abs_norm"]

cmp2[["fc_hz", "social_wimp_norm", "autoeq_abs_norm", "mean_norm"]].head()


,fc_hz,social_wimp_norm,autoeq_abs_norm,mean_norm
0,20.0,0.715676,0.000000,0.357838
1,50.0,0.880835,0.557351,0.719093
2,83.0,1.000000,0.335877,0.667938
3,120.0,0.981146,0.196349,0.588748
4,161.0,0.927502,0.330267,0.628884


## Сравнение кривых важности

In [7]:
plot_df = pd.concat([
    cmp2[["fc_hz", "social_wimp_norm"]].rename(columns={"social_wimp_norm": "value"}).assign(series="SocialFX (norm): mean(|gain|*consistency)"),
    cmp2[["fc_hz", "autoeq_abs_norm"]].rename(columns={"autoeq_abs_norm": "value"}).assign(series="AutoEQ (norm): mean(|gain|)"),
    cmp2[["fc_hz", "mean_norm"]].rename(columns={"mean_norm": "value"}).assign(series="Mean curve (0.5 SocialFX + 0.5 AutoEQ)"),
], ignore_index=True)

fig = px.line(
    plot_df, x="fc_hz", y="value", color="series",
    title="SocialFX vs AutoEQ: normalized importance by frequency (shape comparison)"
)
fig.update_xaxes(type="log", title="Frequency (Hz, log)")
fig.update_yaxes(title="Normalized importance (0..1)", rangemode="tozero")

# ISO vertical markers
for f in ISO_STD:
    fig.add_vline(x=float(f), line_width=1, opacity=0.22)

# 16k highlight
fig.add_vline(x=16000, line_width=2, opacity=0.55)

fig.show()


На графике отображены три кривые:

- SocialFX (нормализованная)
- AutoEQ (нормализованная)
- усреднённая кривая

Также на графике отмечены стандартные частоты ISO и отдельно выделена область 16 kHz.

Наблюдения:

- низкие и средние частоты демонстрируют хорошее согласие между SocialFX и AutoEQ
- в области presence (примерно 2–8 kHz) наблюдается частичное расхождение подходов
- выше 10 kHz в данных AutoEQ отсутствуют явные параметрические корректировки,
  что связано с ограничениями измерений и алгоритмического подхода,
  а не с отсутствием перцептивной значимости этой области
- в SocialFX область выше 10 kHz присутствует,
  но характеризуется малыми и аккуратными корректировками

Таким образом, зона выше 10 kHz рассматривается как перцептивно важная,
но слабо выраженная в параметрических данных AutoEQ,
что обосновывает использование стандартов и априорных знаний
при выборе частот в этой области.

## Кластеризация частот (KMeans)

Для выбора частот эквалайзера используется KMeans-кластеризация.

Кластеризация проводится в пространстве:

- log10 частоты
- нормализованная важность

Цель кластеризации:

- сгруппировать частоты с похожим поведением
- определить центры кластеров как кандидаты частот EQ

Кластеризация выполняется отдельно для:

- SocialFX
- AutoEQ
- усреднённой кривой

In [8]:
def kmeans_freq_centers(df, importance_col, k=8, random_state=0):
    """
    df: must have columns fc_hz, importance_col
    Returns:
      centers_fc: array of k center frequencies (Hz) sorted
      df_out: df with cluster labels
    """
    tmp = df[["fc_hz", importance_col]].dropna().copy()
    tmp = tmp[tmp["fc_hz"] > 0]

    X = np.column_stack([
        np.log10(tmp["fc_hz"].values.astype(float)),
        tmp[importance_col].values.astype(float)
    ])

    weights = tmp[importance_col].values.astype(float)
    # avoid all-zero weights
    weights = np.clip(weights, 1e-9, None)

    km = KMeans(n_clusters=k, random_state=random_state, n_init="auto")
    labels = km.fit_predict(X, sample_weight=weights)

    tmp["cluster"] = labels

    # cluster center in log-frequency dimension
    centers_log_fc = km.cluster_centers_[:, 0]
    centers_fc = np.sort(10 ** centers_log_fc)

    return centers_fc, tmp, km

K = 12
cent_social, lab_social, km_social = kmeans_freq_centers(cmp2, "social_wimp_norm", k=K)
cent_auto,   lab_auto,   km_auto   = kmeans_freq_centers(cmp2, "autoeq_abs_norm",  k=K)
cent_mean,   lab_mean,   km_mean   = kmeans_freq_centers(cmp2, "mean_norm",        k=K)

cent_social, cent_auto, cent_mean


(array([   20.        ,    50.        ,    83.        ,   120.        ,
          161.        ,   254.04821237,   383.        ,   487.2218461 ,
         1583.47576205,  3389.64822524,  8568.99516586, 15954.02569499]),
 array([   49.99999992,    95.09189462,   200.04376241,   443.86399845,
          798.06715519,  1485.98256107,  1875.        ,  2368.08086816,
         4365.36347746,  4392.        ,  9543.86885219, 16591.86737841]),
 array([   20.        ,    50.        ,    83.        ,   159.51029456,
          308.56272462,   622.04351031,  1613.61827835,  2387.71300317,
         3496.61233216,  4392.        ,  6934.26077273, 14932.81571757]))

## График усреднённой важности частот с центрами KMeans

In [9]:
fig = px.line(
    cmp2, x="fc_hz", y="mean_norm",
    title=f"Mean curve with KMeans centers (k={K})"
)
fig.update_xaxes(type="log", title="Frequency (Hz, log)")
fig.update_yaxes(title="Mean normalized importance (0..1)", rangemode="tozero")

for f in cent_mean:
    fig.add_vline(x=float(f), line_width=2, opacity=0.4)

# ISO + 16k
for f in ISO_STD:
    fig.add_vline(x=float(f), line_width=1, opacity=0.18)
fig.add_vline(x=16000, line_width=2, opacity=0.6)
fig.add_vline(x=1000, line_width=2, opacity=0.6)
fig.add_vline(x=12000, line_width=2, opacity=0.6)
fig.add_vline(x=14000, line_width=2, opacity=0.6)


fig.show()


В отличие от предыдущего графика, который сравнивал SocialFX и AutoEQ,
здесь показана только усреднённая кривая важности частот.
Она отражает, в каких зонах спектра эквализация происходит чаще всего.

Синяя линия:
- средняя нормализованная важность частот
- показывает распределение внимания по спектру
- не является EQ-кривой и не задаёт усиление

Вертикальные толстые линии:
- центры кластеров KMeans
- каждая линия соответствует зоне спектра с похожим поведением
- интерпретируются как кандидаты частот эквалайзера

Тонкие вертикальные линии:
- стандартные октавные частоты ISO
- используются как ориентир и для проверки согласованности

Частота 16 kHz добавлена отдельно как психоакустический якорь,
поскольку в данных AutoEQ отсутствуют корректировки выше 10 kHz.

Данный график представляет собой переход
от анализа данных к проектированию эквалайзера.

## Почему добавляем якорь 1 kHz

Точка около 1 kHz рассматривается как "контрольная" (anchor band).

По данным (mean curve) её важность низкая, то есть:
- пользователи и алгоритмы редко делают выраженную коррекцию в этой области
- это удобная опорная зона для стабилизации генерации эквалайзеров

В генераторе искусственных пользователей 1 kHz используется как полоса,
которую важно НЕ "задирать":
- высокая вероятность оставить gain близким к 0
- малая дисперсия значений gain
- полоса фиксируется в финальном наборе частот независимо от KMeans

## Почему фиксируем полосы 12/14/16 kHz

В области выше 10 kHz KMeans ведёт себя нестабильно, потому что:
- в AutoEQ отсутствуют явные параметры выше 10 kHz
- в SocialFX корректировки в зоне "air" имеют малую амплитуду, поэтому метрика важности низкая
- при малом числе точек центры кластеров сильно зависят от деталей сетки

Поэтому используется гибридная стратегия:
- до 10 kHz частоты выбираются data-driven (KMeans по важности)
- выше 10 kHz фиксируются "air bands" 12/14/16 kHz как проектные якоря

Это улучшает эргономику эквалайзера и стабильность генерации синтетических пользователей,
при этом соответствует практическим сценариям "air/sparkle" коррекции.

## Частотные зоны интерпретируются следующим образом:

- 20–50 Hz: Sub-bass
- 50–90 Hz: Bass
- 90–200 Hz: Upper bass
- 200–400 Hz: Low mids (mud)
- 400–800 Hz: Mids / boxy
- ~1 kHz: Core mids (anchor)
- 1–2 kHz: Upper mids
- 2–4 kHz: Presence
- 4–6 kHz: High presence
- 6–8 kHz: Brilliance
- 8–10 kHz: Treble
- 10–12 kHz: Air (low)
- 12–16 kHz: Air / sparkle



## Вывод по работе KMeans и функции merge_close_freqs_keep_anchors

KMeans-кластеризация применяется для выявления основных зон спектра,
в которых эквализация происходит наиболее активно.
Результатом KMeans является ограниченный набор центров кластеров,
который отражает "скелет" частотной структуры эквалайзера.

Пример центров KMeans (k = 12):
- около 20, 50, 80, 160, 300, 600 Hz
- около 1.6k, 2.4k, 3.5k, 4.4k Hz
- около 7k и 15k Hz

Эти частоты представляют собой центры спектральных зон,
а не полный набор полос эквалайзера.

In [10]:
def merge_close_freqs_keep_anchors(freqs_hz, anchors_hz, min_semitones=2.5, anchor_tol_hz=10.0):
    freqs = np.sort(np.asarray(freqs_hz, float))
    anchors = np.asarray(anchors_hz, float)

    if freqs.size == 0:
        return np.sort(np.unique(anchors))

    kept = [freqs[0]]

    for f in freqs[1:]:
        prev = kept[-1]
        semis = 12.0 * np.log2(f / prev)

        if semis >= min_semitones:
            kept.append(f)
        else:
            # если текущая частота близка к якорю - заменим предыдущую на неё
            if np.any(np.isclose(f, anchors, atol=anchor_tol_hz, rtol=0.0)):
                kept[-1] = f

    # гарантируем, что якоря присутствуют
    out = np.unique(np.round(np.concatenate([kept, anchors]), 6))
    out = np.sort(out)
    return out

ANCHORS = [1000.0, 12000.0, 14000.0, 16000.0]

cand = np.concatenate([cent_social, cent_auto, cent_mean])
cand = cand[cand < 10000.0]
cand = np.concatenate([cand, ANCHORS])

final_freqs = merge_close_freqs_keep_anchors(cand, ANCHORS, min_semitones=2.5, anchor_tol_hz=20.0)
final_freqs


array([   20.      ,    50.      ,    83.      ,   120.      ,
         159.510295,   200.043762,   254.048212,   308.562725,
         383.      ,   443.863998,   622.04351 ,   798.067155,
        1000.      ,  1485.982561,  1875.      ,  2368.080868,
        3389.648225,  4365.363477,  6934.260773,  8568.995166,
       12000.      , 14000.      , 16000.      ])

### Откуда появляются дополнительные частоты (120, 200, 383, 443 и др.)

Дополнительные частоты появляются не из одного KMeans,
а в результате объединения нескольких источников:

- центры KMeans, рассчитанные по SocialFX
- центры KMeans, рассчитанные по AutoEQ
- центры KMeans, рассчитанные по усреднённой кривой

Все эти центры объединяются в единый список кандидатов частот.
Таким образом, финальный набор содержит не только "скелет" зон,
но и более детализированные точки внутри этих зон.

Это позволяет:
- сохранить структуру спектра, найденную KMeans
- повысить разрешение эквалайзера внутри важных областей
- использовать более гибкую и реалистичную сетку частот

---

### Роль функции merge_close_freqs_keep_anchors

Функция merge_close_freqs_keep_anchors выполняет постобработку списка
кандидатных частот и решает три задачи:

1) Удаляет слишком близкие частоты
   Если расстояние между соседними частотами меньше заданного порога
   (min_semitones), они считаются избыточными и схлопываются.

2) Сохраняет якорные частоты
   Частоты-якоря (например 1 kHz и 12/14/16 kHz) сохраняются независимо
   от плотности сетки и результатов KMeans.

3) Формирует финальную рабочую сетку
   Итоговый набор частот является гибридом:
   - data-driven структуры (KMeans до 10 kHz)
   - проектных решений и психоакустических якорей (выше 10 kHz)

---

### Интерпретация финального результата

Финальный список частот не является прямым выводом KMeans.
Он представляет собой:

- структурированный набор спектральных зон (из KMeans)
- дополненный частотами внутри зон для более точного контроля
- стабилизированный якорными точками в чувствительных областях

Таким образом:
- KMeans отвечает за архитектуру эквалайзера
- merge_close_freqs_keep_anchors отвечает за его практическую реализацию

Это разделение позволяет совместить
статистическую обоснованность и инженерный контроль.

In [11]:
import plotly.express as px

# финальные полосы
bands = final_freqs

fig = px.line(
    cmp2,
    x="fc_hz",
    y="mean_norm",
    title="Mean importance curve with final EQ bands"
)

fig.update_xaxes(
    type="log",
    title="Frequency (Hz, log scale)"
)

fig.update_yaxes(
    title="Mean normalized importance (0..1)",
    rangemode="tozero"
)

# финальные полосы EQ
for f in bands:
    fig.add_vline(
        x=float(f),
        line_width=2,
        opacity=0.5
    )

# ISO ориентиры (тонкие)
for f in ISO_STD:
    fig.add_vline(
        x=float(f),
        line_width=1,
        opacity=0.2
    )

fig.show()


## Финальные полосы эквалайзера на кривой важности

На графике показана усреднённая нормализованная кривая важности частот
(SocialFX + AutoEQ), а также финальный набор полос эквалайзера.

Вертикальные линии соответствуют итоговым частотам,
полученным после объединения KMeans и постобработки
с сохранением якорных частот.

График демонстрирует, что:
- полосы располагаются в зонах повышенной важности
- каждая полоса покрывает устойчивую область спектра, а не локальный пик
- якорные частоты (например 1 kHz и 12/14/16 kHz) обеспечивают стабильность
  и контроль в чувствительных областях

Таким образом, финальная сетка полос является
гибридом data-driven анализа и осознанных проектных решений.


In [12]:
def nearest_in_set(targets, pool):
    pool = np.asarray(pool, float)
    out = []
    for t in targets:
        i = np.argmin(np.abs(pool - t))
        near = pool[i]
        # error in semitones
        err_semi = 12.0 * np.log2(near / t)
        out.append((t, near, err_semi))
    return pd.DataFrame(out, columns=["iso_hz", "nearest_hz", "err_semitones"])

std_cmp = nearest_in_set(ISO_STD, final_freqs)
std_cmp


,iso_hz,nearest_hz,err_semitones
0,31.5,20.000000,-7.864222
1,63.0,50.000000,-4.001085
2,125.0,120.000000,-0.706724
3,250.0,254.048212,0.278091
4,500.0,443.863998,-2.061725
5,1000.0,1000.000000,0.000000
6,2000.0,1875.000000,-1.117313
7,4000.0,4365.363477,1.513221
8,8000.0,8568.995166,1.189512
9,16000.0,16000.000000,0.000000


## Интерпретация параметра sigma и типов пользователей

В данном коде параметр sigma (σ) используется как мера общей амплитуды
эквализации и отражает, насколько сильно в среднем изменяются полосы
эквалайзера в dB.

Значение sigma рассчитывается по распределению значений gain_db
(без нулевых значений) и описывает характерный масштаб коррекций.

In [13]:
def gain_sigma(df, col="gain_db"):
    g = df[col].dropna().astype(float).values
    g = g[g != 0]
    return float(np.std(g)), float(np.mean(g)), float(np.median(g))

sigma_social, mean_social, med_social = gain_sigma(eq_long_w, "gain_db")
sigma_auto,   mean_auto,   med_auto   = gain_sigma(autoeq, "gain_db")

ratio = sigma_auto / (sigma_social + 1e-12)

print("SocialFX gain sigma:", sigma_social, "mean:", mean_social, "median:", med_social)
print("AutoEQ   gain sigma:", sigma_auto,   "mean:", mean_auto,   "median:", med_auto)
print("Sigma ratio (AutoEQ/SocialFX):", ratio)

# Suggested alpha values for synthetic users
alphas = pd.DataFrame({
    "user_type": ["cautious", "normal", "aggressive"],
    "alpha": [0.7, 1.0, 1.5],
    "sigma_user": [0.7*sigma_social, 1.0*sigma_social, 1.5*sigma_social]
})
alphas


SocialFX gain sigma: 1.0 mean: 3.6195358796556827e-19 median: -0.03851071941354675
AutoEQ   gain sigma: 4.092476901179697 mean: 0.1018402962593855 median: 0.3
Sigma ratio (AutoEQ/SocialFX): 4.092476901175604


,user_type,alpha,sigma_user
0,cautious,0.7,0.7
1,normal,1.0,1.0
2,aggressive,1.5,1.5


На основе базового значения sigma вводится параметр alpha,
который задаёт тип пользователя:

- cautious (alpha = 0.7)
  Аккуратный пользователь, предпочитающий минимальные корректировки.

- normal (alpha = 1.0)
  Типичный пользователь, чьё поведение соответствует среднему
  уровню эквализации в данных SocialFX.

- aggressive (alpha = 1.5)
  Пользователь, склонный к более выраженным изменениям эквалайзера.

Параметр sigma_user определяется как произведение sigma и alpha
и используется для масштабирования амплитуды коррекций
при генерации синтетических эквалайзерных профилей.

Важно отметить, что sigma не влияет на выбор частот эквалайзера.
Архитектура полос определяется отдельно (через кривую важности,
кластеризацию и якорные частоты).

Таким образом:
- частоты и их роль задают структуру эквалайзера
- sigma и alpha задают стиль и интенсивность его использования

## Статистика направлений эквализации (boost vs cut)

В данной таблице представлена статистика направлений эквализации
по каждой частоте в датасете SocialFX.

Для каждой частоты рассчитано:
- сколько раз она использовалась (n_total)
- доля усилений и ослаблений (p_boost / p_cut)
- средняя величина усиления и ослабления (mean_boost / mean_cut)

In [14]:
def boost_cut_stats(df):
    d = df.dropna(subset=["fc_hz", "gain_db"]).copy()
    d = d[d["gain_db"] != 0]

    grp = d.groupby("fc_hz")

    total = grp.size().rename("n_total")
    n_boost = grp.apply(lambda x: (x["gain_db"] > 0).sum()).rename("n_boost")
    n_cut   = grp.apply(lambda x: (x["gain_db"] < 0).sum()).rename("n_cut")

    p_boost = (n_boost / total).rename("p_boost")
    p_cut   = (n_cut / total).rename("p_cut")

    mean_boost = (d[d["gain_db"] > 0].groupby("fc_hz")["gain_db"].mean()).rename("mean_boost")
    mean_cut   = (d[d["gain_db"] < 0].groupby("fc_hz")["gain_db"].mean()).rename("mean_cut")  # negative

    out = pd.concat([total, n_boost, n_cut, p_boost, p_cut, mean_boost, mean_cut], axis=1).reset_index()
    return out.sort_values("fc_hz")

social_stats = boost_cut_stats(eq_long_w)
social_stats.to_csv("../../datasets/socialfx_analysis/socialfx_boost_cut_stats.csv", index=False)
social_stats


C:\Users\makcc\AppData\Local\Temp\ipykernel_26328\1875739653.py:8: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

C:\Users\makcc\AppData\Local\Temp\ipykernel_26328\1875739653.py:9: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,fc_hz,n_total,n_boost,n_cut,p_boost,p_cut,mean_boost,mean_cut
0,20.0,1595,844,751,0.529154,0.470846,1.169733,-1.057404
1,50.0,1595,858,737,0.537931,0.462069,1.240518,-1.139326
2,83.0,1595,882,713,0.552978,0.447022,1.271045,-1.216854
3,120.0,1595,884,711,0.554232,0.445768,1.257726,-1.188433
4,161.0,1595,883,712,0.553605,0.446395,1.232883,-1.135992
5,208.0,1595,880,715,0.551724,0.448276,1.211325,-1.078667
6,259.0,1595,894,701,0.560502,0.439498,1.157108,-1.040927
7,318.0,1595,904,691,0.566771,0.433229,1.108777,-1.006909
8,383.0,1595,899,696,0.563636,0.436364,0.987956,-0.890151
9,455.0,1595,870,725,0.545455,0.454545,0.847278,-0.751753


Результаты показывают, что поведение пользователей
существенно зависит от частотной области:

- низкие частоты чаще усиливаются и с большей амплитудой
- область около 1 kHz является нейтральной и наиболее стабильной
- верхние частоты используются осторожно и чаще ослабляются

Анализ проводится без введения якорных частот,
так как целью является описание реального пользовательского поведения,
а не проектирование или стабилизация эквалайзерной модели.

Таким образом, выявленные закономерности
являются прямым отражением данных SocialFX.

In [49]:
from pathlib import Path
import numpy as np
import pandas as pd

OUTDIR = Path("./outputs")
OUTDIR.mkdir(parents=True, exist_ok=True)

def save_freqs(path: Path, freqs: np.ndarray):
    pd.DataFrame({"fc_hz": np.asarray(freqs, float)}).to_csv(path, index=False)

def save_series_on_f40(path: Path, f40: np.ndarray, y: np.ndarray, col: str):
    df = pd.DataFrame({"fc_hz": np.asarray(f40, float), col: np.asarray(y, float)})
    df.to_csv(path, index=False)

In [47]:
OUTDIR.resolve()

WindowsPath('C:/Users/makcc/PycharmProjects/EarLoop/research/socialfx&autoeq/outputs')

In [50]:
# 1) SocialFX 40-band статистика (источник истины для сравнения)
# Требуется: social_stats
social_stats_path = OUTDIR / "social_stats_40.csv"
social_stats.to_csv(social_stats_path, index=False)

# 2) Частоты компактной сетки
# Требуется: final_freqs
save_freqs(OUTDIR / "freqs_final.csv", final_freqs)

print("Saved artifacts to:", OUTDIR.resolve())
print("Files:", [p.name for p in sorted(OUTDIR.glob("*"))])


Saved artifacts to: C:\Users\makcc\PycharmProjects\EarLoop\research\socialfx&autoeq\outputs
Files: ['freqs_final.csv', 'social_stats_40.csv']
